In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from enum import Enum
import os
import nfl_data_py as nfl
import praw
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
class LeagueType(Enum):
    REDRAFT = 1
    DYNASTY = 2

In [9]:
def get_ranking(player_name, league_type, league_size, ppr_value):
    league_type_string = ""

    if league_type == LeagueType.REDRAFT:
        league_type_string = "redraft"
    elif league_type == LeagueType.DYNASTY:
        league_type_string = "dynasty"

    # read from csv file python
    df = pd.read_csv(f'../fantasy_calc_rankings/{league_type_string}_{ppr_value}_{league_size}.csv', sep=';')
    df = df[df['name'] == player_name]

    # get the value and overallRank from the df
    value = df['value'].iloc[0]
    overallRank = df['overallRank'].iloc[0]

    return value, overallRank
    
get_ranking('Joe Mixon', LeagueType.REDRAFT, 12, 1)

(8207, 8)

In [21]:
from sleeper_wrapper import Stats
import nfl_data_py as nfl
import json
import datetime

def find_player_name(user_prompt_text, player_list):
    # TODO: handle complex scandanavian names with accents
    pattern = r'\b(?:' + '|'.join(re.escape(name) for name in player_list) + r')\b'
    # Search for names in the prompt using the regex pattern, ignoring case
    matches = re.findall(pattern, user_prompt_text, flags=re.IGNORECASE)

    # Return names as they originally appear in the names list
    return [name for name in player_list if
            any(re.match(re.escape(name), match, re.IGNORECASE) for match in matches)]

# def scrape_reddit(player_name, n_rows=10, subreddit_name='nba'):
#     player_name = player_name.lower()
#     reddit = praw.Reddit(
#         client_id="BrA1seVpmeQqXQWOIxnTdA",
#         client_secret="o3Oj5eEIG8YAQZ-0NPVqE9ZkyyET3A",
#         password="39clues",
#         user_agent="fantasyfootballdata",
#         username="joindaclub",
#     )

#     subreddit_name = subreddit_name

#     res = ""
#     res += f'---------- Recent news from reddit for {player_name} ----------\n'
#     for submission in reddit.subreddit(subreddit_name).search(player_name, sort='top', time_filter='week',
#                                                               limit=n_rows):
#         temp_res = " "
#         temp_res += f"Title: {submission.title}\n"
#         # TODO: get content
#         # temp_res += f"Content: {submission.selftext}\n"  # too unfiltered
#         temp_res += f"Comments: {submission.num_comments}\n"
#         temp_res += f"Date: {datetime.datetime.fromtimestamp(submission.created)}\n"
#         temp_res += "\n-----------------\n"

#         if player_name in temp_res.lower():
#             res += temp_res

#     res += '-----------------------------------------------\n'

#     return res

class NFLInterface:
    """
    NFL interface.
    """

    def __init__(self, league_type=LeagueType.REDRAFT, league_size=12, ppr_value=1):
        self.stats = nfl.import_weekly_data([2024])
        self.all_players_list = self.stats["player_display_name"].unique().tolist()
        self.league_type = league_type
        self.league_size = league_size
        self.ppr_value = ppr_value

        if self.league_type == LeagueType.REDRAFT:
            self.league_type_string = "redraft"
        elif self.league_type == LeagueType.DYNASTY:
            self.league_type_string = "dynasty"

        assert self.stats is not None
        assert self.all_players_list is not None

    # n_rows = number of previous weeks stats to give GPT
    def get_nfl_stats(self, player_name, n_rows=10):
        player_stats = self.stats[self.stats["player_display_name"] == player_name]
        first_n_rows = player_stats.to_dict(orient='records')
        first_n_rows.reverse()
        first_n_rows = first_n_rows[:n_rows]
        keys_to_keep = ['recent_team', 'position', 'week', 'opponent_team', 'fantasy_points', 'passing_yards', 'passing_tds', 'interceptions', 'rushing_yards', 'rushing_tds', 'receptions', 'receiving_yards', 'receiving_tds', 'fantasy_points_ppr']
        print(first_n_rows)
        stats_string = '\n'
        stats_string += f'---------- Recent Stats for {player_name} ----------\n'
        for elem in first_n_rows:
            elem = {k: elem[k] for k in keys_to_keep}
            stats_string += json.dumps(elem)
            stats_string += '\n'
        stats_string += '-----------------------------------------------\n'
        return stats_string
    
    # gets a ranking of a player 
    def get_ranking(self, player_name):
        # read from csv file python
        df = pd.read_csv(f'fantasy_calc_rankings/{self.league_type_string}_{self.ppr_value}_{self.league_size}.csv', sep=';')
        df = df[df['name'] == player_name]

        # get the value and overallRank from the df
        value = df['value'].iloc[0]
        overallRank = df['overallRank'].iloc[0]

        return value, overallRank

    def get_player_projected_points(player_name, player_data, season_type, season, week, scoring_format="ppr"):
        """Retrieve the projected points for a specific player for the upcoming match."""
        stats = Stats()
        week_projections = stats.get_week_projections(season_type, season, week)

        # Find player ID from the name
        player_id = next((pid for pid, pdata in player_data.items() if pdata.get("full_name").lower() == player_name.lower()), None)

        if not player_id:
            return f"Player '{player_name}' not found."

        projected_points = week_projections.get(str(player_id), {}).get(f"pts_{scoring_format}", 0)
        return projected_points

    def get_player_total_projected_points(player_name, player_data, season_type, season, current_week, total_weeks=17, scoring_format="ppr"):
        """Retrieve the total projected points for a player for the rest of the season."""
        stats = Stats()
        total_projected_points = 0

        # Find player ID from the name
        player_id = next((pid for pid, pdata in player_data.items() if pdata.get("full_name").lower() == player_name.lower()), None)

        if not player_id:
            return f"Player '{player_name}' not found."

        for week in range(current_week, total_weeks + 1):
            week_projections = stats.get_week_projections(season_type, season, week)
            total_projected_points += week_projections.get(str(player_id), {}).get(f"pts_{scoring_format}", 0)

        return total_projected_points

    def find_player_name(self, user_prompt_text):
        return find_player_name(user_prompt_text, self.all_players_list)

    def build_prompt(self, user_input, verbose=False):

        # add context window, league team, opponent team etc
        # based on query, tell GPT if it needs tools it has a get_ranking and get_nfl_stats (and i can add a get_nfl_news)
        names_found = self.find_player_name(user_input)

        build_prompt = ""

        build_prompt += f"User question: {user_input}\n"
        # reddit_res = scrape_reddit(player_name=player_name, n_rows=10, subreddit_name='nfl')
        # build_prompt += reddit_res

        if verbose:
            print(build_prompt)

        return build_prompt

    def test_interface(self):
        example_prompt = "Will Patrick Mahomes score more than 2 TDs next game?"
        res = self.build_prompt(example_prompt, verbose=False)
        print(res)


nfl_interface = NFLInterface()
nfl_interface.test_interface()
nfl_interface.get_nfl_stats("Patrick Mahomes", n_rows=10)

Downcasting floats.
User question: Will Patrick Mahomes score more than 2 TDs next game?

[{'player_id': '00-0033873', 'player_name': 'P.Mahomes', 'player_display_name': 'Patrick Mahomes', 'position': 'QB', 'position_group': 'QB', 'headshot_url': 'https://static.www.nfl.com/image/upload/f_auto,q_auto/league/qrdbvu4iqy7wkqg8jq8d', 'recent_team': 'KC', 'season': 2024, 'week': 13, 'season_type': 'REG', 'opponent_team': 'LV', 'completions': 26, 'attempts': 46, 'passing_yards': 306.0, 'passing_tds': 1, 'interceptions': 0.0, 'sacks': 5.0, 'sack_yards': 40.0, 'sack_fumbles': 1, 'sack_fumbles_lost': 0, 'passing_air_yards': 385.0, 'passing_yards_after_catch': 145.0, 'passing_first_downs': 14.0, 'passing_epa': 1.5744423866271973, 'passing_2pt_conversions': 0, 'pacr': 0.7948051691055298, 'dakota': 0.02646033465862274, 'carries': 2, 'rushing_yards': 4.0, 'rushing_tds': 0, 'rushing_fumbles': 0.0, 'rushing_fumbles_lost': 0.0, 'rushing_first_downs': 0.0, 'rushing_epa': -0.05619031563401222, 'rushing_

'\n---------- Recent Stats for Patrick Mahomes ----------\n{"recent_team": "KC", "position": "QB", "week": 13, "opponent_team": "LV", "fantasy_points": 16.639999389648438, "passing_yards": 306.0, "passing_tds": 1, "interceptions": 0.0, "rushing_yards": 4.0, "rushing_tds": 0, "receptions": 0, "receiving_yards": 0.0, "receiving_tds": 0, "fantasy_points_ppr": 16.639999389648438}\n{"recent_team": "KC", "position": "QB", "week": 12, "opponent_team": "CAR", "fantasy_points": 28.760000228881836, "passing_yards": 269.0, "passing_tds": 3, "interceptions": 0.0, "rushing_yards": 60.0, "rushing_tds": 0, "receptions": 0, "receiving_yards": 0.0, "receiving_tds": 0, "fantasy_points_ppr": 28.760000228881836}\n{"recent_team": "KC", "position": "QB", "week": 11, "opponent_team": "BUF", "fantasy_points": 15.84000015258789, "passing_yards": 196.0, "passing_tds": 3, "interceptions": 2.0, "rushing_yards": 0.0, "rushing_tds": 0, "receptions": 0, "receiving_yards": 0.0, "receiving_tds": 0, "fantasy_points_ppr

In [2]:
pd.read_parquet('parquet/WDIS_WR_qa_pairs.parquet')

,thread_title,thread_url,question,question_author,answer,answer_author
0,Official: [WDIS WR] - Sun Afternoon 12/01/2024,https://www.reddit.com/r/fantasyfootball/comme...,Addison or Deebo \n\n.5ppr,CrimsonTyphoon0613,Addison.,Glakos
1,Official: [WDIS WR] - Sun Afternoon 12/01/2024,https://www.reddit.com/r/fantasyfootball/comme...,Addison or Deebo \n\n.5ppr,CrimsonTyphoon0613,Same question add Mooney. Currently going with...,Incara1010
2,Official: [WDIS WR] - Sun Afternoon 12/01/2024,https://www.reddit.com/r/fantasyfootball/comme...,Addison or Deebo \n\n.5ppr,CrimsonTyphoon0613,Deebo,PuzzlingSquirrel
3,Official: [WDIS WR] - Sun Afternoon 12/01/2024,https://www.reddit.com/r/fantasyfootball/comme...,Standard scoring - Mooney vs Chargers or Jenni...,littleseizure,Mooney for sure.,ATLdigi
4,Official: [WDIS WR] - Sun Afternoon 12/01/2024,https://www.reddit.com/r/fantasyfootball/comme...,"Amari cooper, Legette, or Demario? 1/2 PPR",HawaiianPunching,I'd go with Cooper,crappykillaonariva
...,...,...,...,...,...,...
997,Official: [WDIS WR] - Tue Evening 11/26/2024,https://www.reddit.com/r/fantasyfootball/comme...,12 Team PPR\n\n\nZ Flowers vs Phi\n\n\nT Higgi...,jabroni716,Zay slightly. \n\nEagles defence is playing gr...,HimothyMozgov3333333
998,Official: [WDIS WR] - Tue Evening 11/26/2024,https://www.reddit.com/r/fantasyfootball/comme...,Jauan Jennings OR a cold weather Tyreek Hill? ...,69_________________,I’d go Tyreek. Is Brock playing this week?,HimothyMozgov3333333
999,Official: [WDIS WR] - Tue Evening 11/26/2024,https://www.reddit.com/r/fantasyfootball/comme...,"Keenan Allen, Jameson Williams or Dowdle?\n\nF...",_In_Need_Of_Support_,I think Jamo. Tough to start 2 bears recs agai...,HimothyMozgov3333333
1000,Official: [WDIS WR] - Tue Evening 11/26/2024,https://www.reddit.com/r/fantasyfootball/comme...,Full PPR: \nWR1: C. Ridley \nWR2: ? \nWho s...,xdokiguess,Jakobi,MattDamonStoudamire


In [4]:
for col in nfl.see_weekly_cols():
    print(col)


nfl.import_ids()

player_id
player_name
player_display_name
position
position_group
headshot_url
recent_team
season
week
season_type
opponent_team
completions
attempts
passing_yards
passing_tds
interceptions
sacks
sack_yards
sack_fumbles
sack_fumbles_lost
passing_air_yards
passing_yards_after_catch
passing_first_downs
passing_epa
passing_2pt_conversions
pacr
dakota
carries
rushing_yards
rushing_tds
rushing_fumbles
rushing_fumbles_lost
rushing_first_downs
rushing_epa
rushing_2pt_conversions
receptions
targets
receiving_yards
receiving_tds
receiving_fumbles
receiving_fumbles_lost
receiving_air_yards
receiving_yards_after_catch
receiving_first_downs
receiving_epa
receiving_2pt_conversions
racr
target_share
air_yards_share
wopr
special_teams_tds
fantasy_points
fantasy_points_ppr


,rotowire_id,birthdate,age,draft_year,ktc_id,fantasypros_id,name,fantasy_data_id,draft_ovr,sleeper_id,...,stats_id,sportradar_id,pfr_id,stats_global_id,cbs_id,draft_round,gsis_id,draft_pick,espn_id,weight
0,17695.0,2001-11-18,23.0,2024.0,1550.0,23084.0,Caleb Williams,NaN,1.0,11560.0,...,40900.0,49d4b627-3f19-49a8-ae33-f539e0fb1f88,WillCa03,0.0,26695775.0,1.0,00-0039918,1.0,4431611.0,215.0
1,17673.0,2002-08-30,22.3,2024.0,1551.0,23046.0,Drake Maye,NaN,3.0,11564.0,...,40881.0,d0c0630e-8925-4b81-a32b-e4146d95f01f,MayeDr00,0.0,26695757.0,1.0,00-0039851,3.0,4431452.0,225.0
2,17692.0,2000-12-18,23.9,2024.0,1552.0,22902.0,Jayden Daniels,NaN,2.0,11566.0,...,40896.0,bab8c65a-9fff-410d-8e34-dc2f636d890e,DaniJa02,0.0,3123337.0,1.0,00-0039910,2.0,4426348.0,210.0
3,16995.0,2000-02-25,24.8,2024.0,1553.0,22910.0,Bo Nix,NaN,12.0,11563.0,...,40875.0,f056c8ed-6ee5-492c-af11-3f7e3d97195c,NixxBo00,0.0,3121193.0,1.0,00-0039732,12.0,4426338.0,217.0
4,17700.0,2000-05-08,24.6,2024.0,1554.0,22973.0,Michael Penix Jr.,NaN,8.0,11559.0,...,40889.0,6499a161-b221-4591-b122-d99ece1be64b,PeniMi00,0.0,2963496.0,1.0,00-0039917,8.0,4360423.0,220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11762,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11763,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11764,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11765,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

urls = [
    "https://fantasycalc.com/redraft-rankings",
    "https://fantasycalc.com/dynasty-rankings"
    
]

# Set up Chrome options
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": "/Users/nickeldime/Code/LLM Class/fantasy-football-llm-assistant/fantasy_calc_rankings"}
chrome_options.add_experimental_option("prefs", prefs)

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the website
driver.get("https://fantasycalc.com/redraft-rankings")

# Find and click the download button
download_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//mat-icon[@mattooltip='Download CSV']"))
)
download_button.click()

# Wait for the download to complete
time.sleep(5)  # Adjust the wait time as needed

# Close the browser
driver.quit()

In [5]:
import json
client = OpenAI(base_url=os.getenv("URL"), api_key=os.getenv("KEY"))



def get_weather(prompt):
    return "The weather in San Francisco today is 72 degrees with a 10% chance of rain."

response = client.chat.completions.create(
    model="llama3.1",
    messages=[
        {"role": "system", "content": "You are a fantasy football bot. Only use the tools provided if needed."},
        {"role": "user", "content": "What is the weather in New York?"},
    ],
    tool_choice="auto",
    tools=[
    {
      "type": "function",
      "function": {
        "name": "get_value",
        "description": "Gets the value of a player, generated by the fantasy football calculator. This can range from 0 to 10000.",
        "parameters": {
          "type": "string",
          "description": "The name of the player"
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_nfl_stats",
        "description": "Gets the recent stats of a player in the NFL.",
        "parameters": {
          "type": "string",
          "description": "The name of the player"
        }
      }
    }
  ]
)

# "parameters": {
#           "type": "object",
#           "properties": {
#             "location": {
#                 "type": "string",
#                 "description": "The city and country, eg. San Francisco, USA"
#             },
#             "format": { "type": "string", "enum": ["celsius", "fahrenheit"] }
#           },
#           "required": ["location", "format"]
#         }

print(response)
# Check for the function call and handle it
if response.choices[0].message.tool_calls:
    # Extract and parse the date argument from the function call
    date_argument = response.choices[0].message.tool_calls[0].function.arguments
    date_argument_dict = json.loads(date_argument)  # Parse the JSON string to a dictionary
    date = date_argument_dict.get("location", None)

    # Call the query_historical_event function with the date
    response = get_weather(date)
    print("Assistant response:", response)
else:
    # If there's no function call, display the assistant's regular response
    print("Assistant response:", response.choices[0].message.content)

NotFoundError: Error code: 404 - {'object': 'error', 'message': 'The model `llama3.1` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}

In [6]:
os.environ["OPENAI_API_KEY"] = os.getenv("KEY")

from llama_stack_client import LlamaStackClient

client = LlamaStackClient(base_url=os.getenv("URL"))

api_key = os.getenv("API_KEY")

client._custom_headers.update({"Authorization": f"Bearer {api_key}"})



response = client.inference.chat_completion(
    model_id="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a fantasy football bot. Use the provided tools to answer the user's question."},
        {"role": "user", "content": "What's the weather in San Francisco today and the likelihood it'll rain?"},
    ]
)
print(response)

AuthenticationError: Unauthorized: Invalid API key

In [7]:
client = OpenAI(base_url=os.getenv("URL"), api_key=os.getenv("KEY"))

PROMPT = """
You are a fantasy football assistant. Please end all your responses with the string "BOB".

For all players, always call both the get_value and get_nfl_stats functions to get the player's value and stats. 
The get_value function returns the value of a player, generated by the fantasy football calculator. This can range from 0 to 10000.

These are the results from the tools:
Calling function: get_value
Arguments: {'player_name': 'Joe Mixon'}
Function output: 8095
Calling function: get_nfl_stats
Arguments: {'n_games': '5', 'player_name': 'Joe Mixon'}
Function output: 
---------- Recent Stats for Joe Mixon ----------
{"recent_team": "HOU", "position": "RB", "week": 13, "opponent_team": "JAX", "fantasy_points": 17.899999618530273, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 101.0, "rushing_tds": 1, "receptions": 4, "receiving_yards": 18.0, "receiving_tds": 0, "fantasy_points_ppr": 21.899999618530273}
{"recent_team": "HOU", "position": "RB", "week": 12, "opponent_team": "TEN", "fantasy_points": 4.5, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 22.0, "rushing_tds": 0, "receptions": 5, "receiving_yards": 23.0, "receiving_tds": 0, "fantasy_points_ppr": 9.5}
{"recent_team": "HOU", "position": "RB", "week": 11, "opponent_team": "DAL", "fantasy_points": 33.29999923706055, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 109.0, "rushing_tds": 3, "receptions": 2, "receiving_yards": 44.0, "receiving_tds": 0, "fantasy_points_ppr": 35.29999923706055}
{"recent_team": "HOU", "position": "RB", "week": 10, "opponent_team": "DET", "fantasy_points": 15.0, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 46.0, "rushing_tds": 1, "receptions": 2, "receiving_yards": 44.0, "receiving_tds": 0, "fantasy_points_ppr": 17.0}
{"recent_team": "HOU", "position": "RB", "week": 9, "opponent_team": "NYJ", "fantasy_points": 16.600000381469727, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 106.0, "rushing_tds": 1, "receptions": 0, "receiving_yards": 0.0, "receiving_tds": 0, "fantasy_points_ppr": 16.600000381469727}
-----------------------------------------------

Calling function: get_value
Arguments: {'player_name': 'Christian Watson'}
Function output: 805
Calling function: get_nfl_stats
Arguments: {'n_games': '5', 'player_name': 'Christian Watson'}
Function output: 
---------- Recent Stats for Christian Watson ----------
{"recent_team": "GB", "position": "WR", "week": 13, "opponent_team": "MIA", "fantasy_points": 6.699999809265137, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 0.0, "rushing_tds": 0, "receptions": 2, "receiving_yards": 67.0, "receiving_tds": 0, "fantasy_points_ppr": 8.699999809265137}
{"recent_team": "GB", "position": "WR", "week": 12, "opponent_team": "SF", "fantasy_points": 0.4000000059604645, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 4.0, "rushing_tds": 0, "receptions": 0, "receiving_yards": 0.0, "receiving_tds": 0, "fantasy_points_ppr": 0.4000000059604645}
{"recent_team": "GB", "position": "WR", "week": 11, "opponent_team": "CHI", "fantasy_points": 15.0, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 0.0, "rushing_tds": 0, "receptions": 4, "receiving_yards": 150.0, "receiving_tds": 0, "fantasy_points_ppr": 19.0}
{"recent_team": "GB", "position": "WR", "week": 9, "opponent_team": "DET", "fantasy_points": 5.699999809265137, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 0.0, "rushing_tds": 0, "receptions": 3, "receiving_yards": 37.0, "receiving_tds": 0, "fantasy_points_ppr": 8.699999809265137}
{"recent_team": "GB", "position": "WR", "week": 8, "opponent_team": "JAX", "fantasy_points": 3.9000000953674316, "passing_yards": 0.0, "passing_tds": 0, "interceptions": 0.0, "rushing_yards": 0.0, "rushing_tds": 0, "receptions": 4, "receiving_yards": 39.0, "receiving_tds": 0, "fantasy_points_ppr": 7.900000095367432}
-----------------------------------------------
"""

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "Should I trade Travis Etienne for Travis Kelce? Respond with 'yes' or 'no'."},
    ],
)

response.choices[0].message.content

'no'

In [1]:
import yaml

def load_benchmark(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

benchmark_file = '../benchmarking/benchmark.yaml'
benchmark_data = load_benchmark(benchmark_file)

def basic_llama(input, expected_answer):
    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3.1-8B-Instruct",
        messages=[
            {"role": "user", "content": input},
        ],
    )

    print(expected_answer, response.choices[0].message.content, expected_answer.upper() in response.choices[0].message.content.upper())

    return expected_answer.upper() in response.choices[0].message.content.upper()

def calculate_accuracy(benchmark_data, test_func):
    total_questions = 0
    correct_answers = 0

    for category, qa_pairs in benchmark_data.items():
        for qa in qa_pairs:
            user_input = qa['question']
            expected_answer = qa['answer']
            result = test_func(user_input, expected_answer)
            if result:
                correct_answers += 1
            total_questions += 1

    accuracy = (correct_answers / total_questions) * 100
    return accuracy

calculate_accuracy(benchmark_data, basic_llama)

NameError: name 'client' is not defined

In [8]:
from agent import NFLAgent

nfl_agent = NFLAgent()

calculate_accuracy(benchmark_data, nfl_agent.test_interface)

Robinson It's a tough decision, but I'd recommend starting Bijan Robinson this week. His recent stats are impressive, with over 100 rushing yards and two touchdowns in one game, and he seems to be the more consistent RB between the two. True
Hill I would recommend starting Hill this week. True
Robinson Robinson True
start I don't have a value for A.J. Brown in my database. Could you please provide more information or clarify the name of the player? False
start I can't provide an answer to this question as I am unable to access information about a bad secondary for Trevor Lawrence. Can you tell me the name of the team he is playing against? False
no You should not trade Christian McCaffrey for Sam Darnold, as Darnold's value is lower than McCaffrey's (if he has a value). True
no No True
no You should not trade D'Andre Swift for a waiver wire kicker. True
yes The value of Travis Etienne is either not available or equal to 0.

You should not trade your backup tight end for Tyreek Hill, as

53.333333333333336